In [1]:
# %matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np 
import pandas as pd
import os
import csv
from resources.conv_learner import *

# required to get individual channels out of image object
from PIL import Image, ImageSequence

### Create labels csv:

Ensure files are labeled according to: "Cell-line-Id + Experiment_ID + _00x.nd2"<br>
Note the Cell-line-Id has '-' delimiters, while all other name-fields have '_' delimiters!

In [3]:
# make csv from images in folder

def csv_from_dir(path, folders, labels_dict, outfile):
    _fnames = []
    if not isinstance(folders, list): folders = [folders]
    print(type(folders), folders)
    for folder in folders: _fnames.extend(read_dir(path, folder)) 
    
    fnames = [[os.path.basename(ims).split('.')[0], # get image basename
                               labels_dict[os.path.basename(ims).split('_')[0]] # get label from labels_dict                              
                              ] for ims in _fnames]
    
    fnames_df = pd.DataFrame(fnames, columns=['Id', 'Targets'])
    uni_fnames = fnames_df.drop_duplicates('Id'); # only unique fnames
    
    print(f'Total unique files: {len(uni_fnames)}')
    return uni_fnames

In [14]:
# manually define labels:
labels = {'NM17-002': 0,
         'Ctrl-AK': 0,
         'GUK1-D': 1, 
         'GUK1-R': 1,
         'MFN2-1': 2,
         'VCP-1': 3}

### Define other arguments:
path = 'datasets/Hirano3D/'
folders = 'Hirano3D_MaxP_GRFB_02_512'
# folders = ['Hirano3D_MaxP_GRFB_02_512', 'Hirano3D_MaxP_GRFB_02_2_512']
outfile = 'Hirano3D_02_all_v1.csv'

In [15]:
# run function:
Hirano3D_df = csv_from_dir(path, folders, labels, outfile)

<class 'list'> ['Hirano3D_MaxP_GRFB_02_512']
Total unique files: 359


In [16]:
### save to csv
Hirano3D_df.to_csv(path + outfile, index=False)

### Generate multi-folder csv:

In [18]:
# add partial path to Id's in csv's

def multi_folder_csv(df_folderpath_dict):
    """
    df_folderpath_dict: dictionary of (at least 2 pairs) of {Partial_folder_path : label_dataframe}, 
    e.g. with label_dataframes containing columns 'Id' and 'Target'.
    
    res: one unified pandas DataFrame with Partial_folder_path added to 'Id's 
    """
    _dict = df_folderpath_dict.copy()
    for partial_path, df in _dict.items():
        for n in tqdm(df.Id, total = len(df.Id), unit="files"):
            df.Id.replace(n, partial_path + n, inplace=True)  
            
    res = list(_dict.values())[0].append(list(_dict.values())[1:])
    
    return res

In [19]:
path = 'datasets/Hirano3D/'
Hirano3D_02_df = pd.read_csv(path + 'Hirano3D_02_all_v1.csv')
Hirano3D_02_2_df = pd.read_csv(path + 'Hirano3D_02_2_all_v1.csv')

df_folderpath_dict = {'Hirano3D_MaxP_GRFB_02_512/':Hirano3D_02_df,
                     'Hirano3D_MaxP_GRFB_02_2_512/': Hirano3D_02_2_df}

In [20]:
mod_df = multi_folder_csv(df_folderpath_dict)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 471/471 [00:00<00:00, 1662.67files/s]


In [21]:
mod_df.to_csv(path + 'multi_folder_Hirano3D_v1.csv', index=False)

### Image checking:

In [61]:
import skimage.external.tifffile as tiff

In [63]:
im = tiff.imread(path + 'test_zproject/' +'GUK1-R_064.tif')
im.shape

(4, 512, 512)

In [ ]:
tiff.imshow(im[0])